In [4]:

# Jupyter Dash setup
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd

from animal_shelter import AnimalShelter

###########################
# Database Connection
###########################

username = "aacuser"
password = "Mytimehoney1337!"

db = AnimalShelter()

# Load initial dataset
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

###########################
# Dashboard App
###########################

app = JupyterDash(__name__)

# Logo
image_filename = 'salvare_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([

    html.Center(html.H1("CS-340 Grazioso Salvare Dashboard")),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', width="300px")),
    html.Center(html.H5("Dashboard by Caleb Sorell")),

    html.Hr(),

    html.Label("Select Rescue Filter:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset (All Animals)', 'value': 'reset'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster/Tracking Rescue', 'value': 'disaster'}
        ],
        value='reset',
        labelStyle={'display': 'block'}
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Hr(),

    html.Div(style={'display': 'flex'}, children=[

        html.Div(id='graph-id', style={'width': '50%'}),

        html.Div(id='map-id', style={'width': '50%'})

    ])
])

###########################
# FILTER CALLBACK
###########################

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):

    if filter_type == 'water':
        query = {
            "breed": {"$regex": "Labrador|Newfoundland", "$options": "i"},
            "age_upon_outcome_in_weeks": {"$lt": 104}
        }

    elif filter_type == 'mountain':
        query = {
            "breed": {"$regex": "German Shepherd|Alaskan Malamute", "$options": "i"},
            "age_upon_outcome_in_weeks": {"$lt": 104}
        }

    elif filter_type == 'disaster':
        query = {
            "breed": {"$regex": "Doberman|Rottweiler", "$options": "i"},
            "age_upon_outcome_in_weeks": {"$lt": 104}
        }

    else:
        query = {}

    dff = pd.DataFrame.from_records(db.read(query))

    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    return dff.to_dict('records')

###########################
# CHART CALLBACK
###########################

@app.callback(
    Output('graph-id', 'children'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_graphs(viewData):

    if viewData is None:
        return []

    dff = pd.DataFrame(viewData)

    fig = px.histogram(
        dff,
        x="breed",
        title="Breed Distribution"
    )

    return [dcc.Graph(figure=fig)]

###########################
# MAP CALLBACK
###########################

@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, selected_rows):

    if not viewData:
        return []

    dff = pd.DataFrame(viewData)

    row = selected_rows[0] if selected_rows else 0

    lat = dff.iloc[row]["location_lat"]
    lon = dff.iloc[row]["location_long"]
    breed = dff.iloc[row]["breed"]
    name = dff.iloc[row]["name"]

    return [

        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[lat, lon],
            zoom=10,
            children=[

                dl.TileLayer(),

                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H3("Animal Name"),
                            html.P(name)
                        ])
                    ]
                )
            ]
        )
    ]

###########################
# RUN APP
###########################

app.run_server(mode='external', debug=True)


Dash app running on https://antennalicense-regarddallas-3000.codio.io/proxy/8050/
